In [3]:
## Libraries and packages needed to run with MP-API
'''
boto3
custodian
emmet-core[all]>=0.39.8
monty==2022.3.12
mpcontribs-client>=5.0.10
mp_api
msgpack
pydantic>=1.8.2
pymatgen>=2022.3.7
pymatgen-analysis-alloys>=0.0.3
typing-extensions==4.1.1
requests==2.27.1
'''
import json
from mp_api.client import MPRester
from mpcontribs.client import Client
from emmet.core.summary import HasProps

In [4]:
## User Input
get_band_gap = False
get_bulk_modulus = True
get_e_electronic = True
get_e_ij_max = True
get_e_ionic = True
get_e_total = True
get_elec_cond = False
get_n = True
get_shear_modulus  = True  
get_therm_cond = False
get_total_magnetization = False
get_total_magnetization_normalized_vol = False
get_total_magnetization_normalized_formula_units = False
get_universal_anisotropy = False

get_mp_ids_contrib = (get_elec_cond and get_therm_cond)


In [5]:
def generate_json(get_band_gap,
                  get_bulk_modulus, 
                  get_e_electronic, 
                  get_e_ij_max, 
                  get_e_ionic, 
                  get_e_total, 
                  get_elec_cond, 
                  get_mp_ids_contrib, 
                  get_n, 
                  get_shear_modulus, 
                  get_therm_cond, 
                  get_total_magnetization, 
                  get_total_magnetization_normalized_formula_units, 
                  get_total_magnetization_normalized_vol, 
                  get_universal_anisotropy):

    if get_mp_ids_contrib:

        client = Client(apikey="uJpFxJJGKCSp9s1shwg9HmDuNjCDfWbM", project="carrier_transport")
        '''
        list_of_query_params = client.available_query_params()
        for item in list_of_query_params:
            print(item)  # print list of available query parameters
        '''
    else:
        client = Client(apikey="uJpFxJJGKCSp9s1shwg9HmDuNjCDfWbM")

    ## Assemble dictionary of values needed for Hashin-Shtrikman analysis
    final_dict = {"mp-ids": [],
                "mp-ids-contrib": [], 
                "formula": [],
                "metal": [],
                "bulk_modulus": [],
                "shear_modulus": [],
                "universal_anisotropy": [],
                "e_total": [],
                "e_ionic": [],
                "e_electronic": [],
                "n": [],
                "e_ij_max": [],
                "therm_cond_300K_low_doping": [],
                "elec_cond_300K_low_doping": []}
    
    fields = ["material_id", "is_stable", "is_metal"]
    if get_band_gap:
        fields.append("band_gap")
    if get_bulk_modulus:
        fields.append("bulk_modulus")
    if get_e_electronic:
        fields.append("e_electronic")
    if get_e_ij_max:
        fields.append("e_ij_max")
    if get_e_ionic:
        fields.append("e_ionic")
    if get_e_total:
        fields.append("e_total")
    if get_n:
        fields.append("n")
    if get_shear_modulus:
        fields.append("shear_modulus")
    if get_total_magnetization:
        fields.append("total_magnetization")
    if get_total_magnetization_normalized_formula_units:
        fields.append("total_magnetization_normalized_formula_units")
    if get_total_magnetization_normalized_vol:
        fields.append("total_magnetization_normalized_vol")
    if get_universal_anisotropy:
        fields.append("universal_anisotropy")

    with MPRester("uJpFxJJGKCSp9s1shwg9HmDuNjCDfWbM") as mpr:
        docs = mpr.materials.summary.search(fields=fields)
        '''
        list_of_available_fields = mpr.materials.summary.available_fields
        for item in list_of_available_fields:
            print(item)  # print list of available field parameters
        '''
    
        #mp_ids = []
        for i, doc in enumerate(docs[0:100]):

            print(f"{i} of {len(docs[0:100])}")

            try:

                mp_id = doc.material_id                           
                query = {"identifier": mp_id}
                my_dict = client.download_contributions(query=query, include=["tables"])[0]
                final_dict["mp-ids"].append(mp_id)    
                final_dict["formula"].append(my_dict["formula"])
                final_dict["metal"].append(my_dict["data"]["metal"])                  
                final_dict["is_stable"].append(doc.is_stable)
                final_dict["is_metal"].append(doc.is_metal) 

                if get_band_gap:
                    final_dict["band_gap"].append(doc.band_gap)
                if get_bulk_modulus:
                    bulk_modulus_voigt = doc.bulk_modulus["voigt"]
                    final_dict["bulk_modulus"].append(bulk_modulus_voigt)
                if get_e_electronic:
                    final_dict["e_electronic"].append(doc.e_electronic)
                if get_e_ij_max:
                    final_dict["e_ij_max"].append(doc.e_ij_max)
                if get_e_ionic:
                    final_dict["e_ionic"].append(doc.e_ionic)
                if get_e_total:
                    final_dict["e_total"].append(doc.e_total)
                if get_n:
                    final_dict["n"].append(doc.n)
                if get_shear_modulus:
                    shear_modulus_voigt = doc.shear_modulus["voigt"]
                    final_dict["shear_modulus"].append(shear_modulus_voigt) 
                if get_total_magnetization:
                    final_dict["total_magnetization"].append(doc.total_magnetization)
                if get_total_magnetization_normalized_formula_units:
                    final_dict["total_magnetization_normalized_formula_units"].append(doc.total_magnetization_normalized_formula_units)
                if get_total_magnetization_normalized_vol:
                    final_dict["total_magnetization_normalized_vol"].append(doc.total_magnetization_normalized_vol)
                if get_universal_anisotropy:
                    final_dict["universal_anisotropy"].append(doc.universal_anisotropy)

                if get_mp_ids_contrib:

                    try:
                        final_dict["mp-ids-contrib"].append(my_dict["identifier"])
                        thermal_cond = my_dict["tables"][7].iloc[2, 1] * 1e-14  # multply by relaxation time, 10 fs
                        elec_cond = my_dict["tables"][5].iloc[2, 1] * 1e-14 # multply by relaxation time, 10 fs   
                        final_dict["therm_cond_300K_low_doping"].append(thermal_cond)
                        final_dict["elec_cond_300K_low_doping"].append(elec_cond)              

                    except:
                        IndexError

            except:
                TypeError
                print("TypeError")
                print(doc.bulk_modulus["voigt"])
                print(doc.e_ij_max)

    with open("final_dict_test.json", "w") as my_file:
        json.dump(final_dict, my_file)

    return

In [6]:
generate_json(get_band_gap,
            get_bulk_modulus, 
            get_e_electronic, 
            get_e_ij_max, 
            get_e_ionic, 
            get_e_total, 
            get_elec_cond, 
            get_mp_ids_contrib, 
            get_n, 
            get_shear_modulus, 
            get_therm_cond, 
            get_total_magnetization, 
            get_total_magnetization_normalized_formula_units, 
            get_total_magnetization_normalized_vol, 
            get_universal_anisotropy)

Retrieving SummaryDoc documents:   0%|          | 0/153235 [00:00<?, ?it/s]

0 of 100
TypeError
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[5], line 87, in generate_json(get_band_gap, get_bulk_modulus, get_e_electronic, get_e_ij_max, get_e_ionic, get_e_total, get_elec_cond, get_mp_ids_contrib, get_n, get_shear_modulus, get_therm_cond, get_total_magnetization, get_total_magnetization_normalized_formula_units, get_total_magnetization_normalized_vol, get_universal_anisotropy)
     86 query = {"identifier": mp_id}
---> 87 my_dict = client.download_contributions(query=query, include=["tables"])[0]
     88 final_dict["mp-ids"].append(mp_id)    

IndexError: list index out of range

During handling of the above exception, another exception occurred:

TypeError                                 Traceback (most recent call last)
Cell In[6], line 1
----> 1 generate_json(get_band_gap,
      2             get_bulk_modulus, 
      3             get_e_electronic,